# Aquire and store historical weather data for Kirkwall Airport from WeatherHQ

https://www.weatherhq.co.uk/weather-station/kirkwall-airport
https://www.vackertvader.se/v%C3%A4derstation/kirkwall-airport
http://archive.vackertvader.se/archive/epoch_observations?station_id=3017&start=1517734899000&end=1549275655000

stationid kirkwall airport - 3017

2999	
0	1549273800000 - timestamp*1000
1	4 - temperature degrees celcius
2	null
3	1002 - pressure mbar
4	3.6 wind speed - m/s
5	null
6	93 - humidity %
7	280 - winddirection degrees
8	null
9	10000 - visibility m
10	37.5 - cloud cover %
11	1036 - cloud height m

In [6]:
from datetime import datetime, date, time
t = datetime(2005, 7, 14, 12, 30)
t.isoformat()

'2005-07-14T12:30:00'

In [12]:
from datetime import datetime
import calendar

d = datetime.utcnow()
unixtime = calendar.timegm(d.utctimetuple())
print (unixtime, d)

1572809255 2019-11-03 19:27:35.205639


In [16]:
def unixtime(d):
    return calendar.timegm(d.utctimetuple())

startdate = datetime(2019,1, 16, 12, 0)
enddate = datetime.utcnow()
print(unixtime(startdate))
print(unixtime(enddate))

1547640000
1572810561


In [21]:
print(str(1000*unixtime(startdate)))

1547640000000


In [22]:
print('http://archive.vackertvader.se/archive/epoch_observations?station_id=3017&start='+str(1000*unixtime(startdate))+'&end='+str(1000*unixtime(enddate)))

http://archive.vackertvader.se/archive/epoch_observations?station_id=3017&start=1547640000000&end=1572810561000


In [67]:
#!/usr/bin/env python3
# this retuen http erro 403forbidden 
# according to https://stackoverflow.com/questions/16627227/http-error-403-in-python-3-web-scraping
# This is probably because of mod_security or some similar server security feature which blocks known spider/bot user agents (urllib uses something like python urllib/3.3.0, it's easily detected). Try setting a known browser user agent with:

import urllib
import json

startdate =datetime(2019, 1, 14, 23, 59, 59)
enddate =datetime.now()
URL = 'http://archive.vackertvader.se/archive/epoch_observations?station_id=3017&start='+str(1000*unixtime(startdate))+'&end='+str(1000*unixtime(enddate))

print(URL)

url = urllib.request.urlopen(URL)
page = url.read()
#data = json.load(page)
print(page)       
#import urllib2
#response = urllib2.urlopen('https://api.instagram.com/v1/tags/pizza/media/XXXXXX')
#html=response.read()
#print html

http://archive.vackertvader.se/archive/epoch_observations?station_id=3017&start=1547510399000&end=1572821188000


HTTPError: HTTP Error 403: Forbidden

In [68]:
from urllib.request import Request, urlopen

req = Request('http://archive.vackertvader.se/archive/epoch_observations?station_id=3017&start='+str(1000*unixtime(startdate))+'&end='+str(1000*unixtime(enddate)), headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
data = json.loads(webpage)

data

In [70]:
print ('from: ', data['from'], 'to: ',data['to'])

from:  1546646399000 to:  1573685188000


In [73]:
import pandas as pd

df = pd.DataFrame.from_records(data['data'])

In [74]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,1561299600000,14.0,None,1024.0,8.2,NaN,82,90.0,None,10000.0,12.5,243.0
1,1561302000000,14.3,None,1021.3,7.2,NaN,81,80.0,None,30000.0,37.5,NaN
2,1561303200000,14.0,None,1024.0,7.2,NaN,82,90.0,None,10000.0,37.5,213.0
3,1561306800000,13.0,None,1024.0,6.7,NaN,87,100.0,None,10000.0,75.0,243.0
4,1561308600000,13.0,None,1024.0,6.7,NaN,87,110.0,None,10000.0,75.0,243.0


In [75]:
df.columns=['timestamp', 'Temp','na','Pressure','WindSpeed','na2','Humidity','WindDirection','na3','Visibility','CloudCover','CloudHeight']

In [76]:
df.head()

,timestamp,Temp,na,Pressure,WindSpeed,na2,Humidity,WindDirection,na3,Visibility,CloudCover,CloudHeight
0,1561299600000,14.0,None,1024.0,8.2,NaN,82,90.0,None,10000.0,12.5,243.0
1,1561302000000,14.3,None,1021.3,7.2,NaN,81,80.0,None,30000.0,37.5,NaN
2,1561303200000,14.0,None,1024.0,7.2,NaN,82,90.0,None,10000.0,37.5,213.0
3,1561306800000,13.0,None,1024.0,6.7,NaN,87,100.0,None,10000.0,75.0,243.0
4,1561308600000,13.0,None,1024.0,6.7,NaN,87,110.0,None,10000.0,75.0,243.0


In [77]:
df = df.drop(['na','na2','na3'], 1)

In [78]:
df.head()

,timestamp,Temp,Pressure,WindSpeed,Humidity,WindDirection,Visibility,CloudCover,CloudHeight
0,1561299600000,14.0,1024.0,8.2,82,90.0,10000.0,12.5,243.0
1,1561302000000,14.3,1021.3,7.2,81,80.0,30000.0,37.5,NaN
2,1561303200000,14.0,1024.0,7.2,82,90.0,10000.0,37.5,213.0
3,1561306800000,13.0,1024.0,6.7,87,100.0,10000.0,75.0,243.0
4,1561308600000,13.0,1024.0,6.7,87,110.0,10000.0,75.0,243.0


In [79]:
df.timestamp = df.timestamp//1000

In [80]:
df.head()

,timestamp,Temp,Pressure,WindSpeed,Humidity,WindDirection,Visibility,CloudCover,CloudHeight
0,1561299600,14.0,1024.0,8.2,82,90.0,10000.0,12.5,243.0
1,1561302000,14.3,1021.3,7.2,81,80.0,30000.0,37.5,NaN
2,1561303200,14.0,1024.0,7.2,82,90.0,10000.0,37.5,213.0
3,1561306800,13.0,1024.0,6.7,87,100.0,10000.0,75.0,243.0
4,1561308600,13.0,1024.0,6.7,87,110.0,10000.0,75.0,243.0


In [85]:
%load_ext sql

ModuleNotFoundError: No module named 'sql'

In [84]:
%sql sqlite:///.databse/vackerWeather.db

UsageError: Line magic function `%sql` not found.
